In [1]:
#1.导入基础库
from csv import reader
from math import exp,pi,sqrt
from random import randrange,seed
import copy
 
#2.读取csv文件和转换数据类型
#读取csv文件
def csv_loader(file):
    dataset=list()
    with open(file,'r') as f:
        csv_reader=reader(f)    
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset
 
#转换属性为浮点数
def str_to_float_converter(dataset):
    dataset=dataset[1:]
    for i in range(len(dataset[0])-1):
        for row in dataset:
            row[i]= float(row[i].strip())
 
#转换标签为整数
def str_to_int_converter(dataset):
    class_values= [row[-1] for row in dataset]
    unique_values= set(class_values)
    converter_dict=dict()
    for i,value in enumerate(unique_values):
        converter_dict[value] = i
    for row in dataset:
        row[-1] = converter_dict[row[-1] ]
 
#3.K折交叉验证拆分数据
def k_fold_cross_validation(dataset,n_folds):
    dataset_splitted=list()
    fold_size= int(len(dataset)/n_folds)
    dataset_copy = list(dataset)
    for i in range(n_folds):
        fold_data = list()
        while len(fold_data) < fold_size:
            index = randrange(len(dataset_copy))
            fold_data.append(dataset_copy.pop(index))
        dataset_splitted.append(fold_data)
    return dataset_splitted
 
#4.计算准确性
def calculate_accuracy(actual,predicted):
    correct_num = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct_num +=1
    accuracy = correct_num/float(len(actual)) *100.0
    return accuracy
 
#5.模型测试
def mode_test(dataset,algo,n_folds,*args):
    dataset_splitted = k_fold_cross_validation(dataset,n_folds)
    scores  = list() 
    for fold in dataset_splitted:
        train = copy.deepcopy(dataset_splitted)
        train.remove(fold)
        train = sum(train, [])
        test =list()
        test = copy.deepcopy(fold)
        predicted = algo(train, test, *args)
        actual = [row[-1] for row in fold]
        accuracy= calculate_accuracy(actual,predicted)
        scores.append(accuracy)
    return scores
 
#6.数据按字典分类和描述
#数据组合成按每一种类进行分类
def split_class(dataset):
    splitted = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if class_value not in splitted:
            splitted[class_value]=list()
        splitted[class_value].append(vector)
    return splitted
 
#计算每一列（x_i)的均值
def calculate_mean(column):
    mean = sum(column)/len(column)
    return mean
 
#计算每一列（x_i)的标准差
def calculate_std(column):
    mean = calculate_mean(column)
    var = sum([(x - mean )**2  for x in column])/float((len(column)-1))
    std = sqrt(var)
    return std
 
#描述数据[(mean，std，len)]
def describe_data(dataset):
    description = [(calculate_mean(column), calculate_std(column),
                   len(column)) for column in zip(*dataset)]
    del description[-1]
    return description
 
#数据按字典分类描述{class:[(mean，std，len)])}
def describe_class(dataset):
    splitted = split_class(dataset)
    descriptions = dict()
    for class_value, rows in splitted.items():
        descriptions[class_value] = describe_data(rows)
    return descriptions
 
#7.计算概率的基础模型
def calculate_probability(x,mean,std):
    exponent = exp(-((x - mean)**2)/(2 *(std**2)))
    probability = (1/(sqrt(2* pi) * std)) *exponent
    return probability
 
#8.计算每一行数据的概率
def calculate_class_probabilities(dataset,row):
    descriptions= describe_class(dataset)
    total = sum([descriptions[label][0][-1] for label in descriptions])
    pribabilities = dict()
    for class_key, class_value  in descriptions.items():
        pribabilities [class_key] = class_value[0][-1]/float(total)
        for i in range(len(class_value)):
            mean,std,count = class_value[i]
            pribabilities [class_key] *= calculate_probability(row[i],mean,std)
    return pribabilities
            
#9.每一行数据中找出最好的标签
def predict(dataset,row):
    pribabilities=calculate_class_probabilities(dataset,row)
    best_label,best_probability =None, -1
    for class_value, probability in pribabilities.items():
        if best_label is None or probability >best_probability:
            best_probability = probability
            best_label= class_value
    return best_label
 
#10.预测测试数据的分类
def naive_bayes(train,test):
    pedictions = list()
    for row in test:
        prediction = predict(train,row)
        pedictions.append(prediction)
    return pedictions
 
#11.运行和参数调整
seed(5)
file='E:\教学\数据挖掘\iris.csv'
dataset= csv_loader(file)
str_to_float_converter(dataset)
dataset=dataset[1:]     
str_to_int_converter(dataset)
n_folds=3
algo=naive_bayes
scores = mode_test(dataset,algo,n_folds)
 
print('The scores of our model are : %s' % scores)
print('The average score of our model is : %.3f%%' % (sum(scores)/float(len(scores))))

The scores of our model are : [98.0, 100.0, 100.0]
The average score of our model is : 99.333%


#### 为什么需要用到K折交叉验证？K值交叉验证的基本思想是什么？

#### 更准确的模型评估
传统的训练集-测试集分割（如80-20划分）可能因单次划分的随机性导致评估结果不稳定。K折交叉验证通过多次划分数据集并取平均结果，显著降低了评估的方差。例如，在鸢尾花数据集（150条样本）中，3折交叉验证会将数据分为3份（每份50条），每次用不同部分测试，最终综合3次结果，使得评估更具代表性。

#### 充分利用有限数据
当数据量较少时（如医学影像或小样本研究），直接划分测试集可能导致训练数据不足。K折交叉验证确保每个样本都被用于训练和测试各一次，最大化数据利用率。例如，3折交叉验证中，每个样本会出现在训练集2次（每次2/3数据）和测试集1次。

#### 检测过拟合倾向
如果模型在不同折上的表现差异很大（如准确率从90%骤降到60%），可能表明模型对某些数据特征过度敏感，存在过拟合风险。这种波动性通过单一划分难以发现。

#### 超参数调优的可靠性
在调整模型参数（如朴素贝叶斯的平滑系数）时，K折交叉验证能更稳定地比较不同参数组合的性能，避免因数据划分偏差导致误选次优参数。
